# Explore SDSSRM-XCS Scatter: Jackknifing to assess $M_{\rm{tot}}$-$T_{\rm{X}}$

This section of the project explores potential reasons and diagnostics of galaxy cluster/group scaling relation scatter using the new SDSSRM-XCS relations constructed in this work from the properties measured in the first paper of this series [(Turner et al. 2024)](https://ui.adsabs.harvard.edu/abs/2025MNRAS.tmp...10T/abstract).

Here we particularly focus on using the common 'jackknife' re-sampling technique in an attempt to identify particular clusters that are the dominant cause of a large scatter measurement for some of our more important scaling relations. Such data points (if they are present) may be genuine measurement outliers, caused by some fitting failure or analysis problem for an individual cluster in the first paper of this series. They may also be clusters that _are_ intrinsically more scattered. 

Scaling relation fits were performed using the XGA interface to the R scaling-relation fitting package LIRA, see the paper for full details.

## Main takeaways

In summary:

* **<span style="color:red">.......</span>**

## Import Statements 

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity
from astropy.cosmology import LambdaCDM
import pickle
import matplotlib
from matplotlib import pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
from tqdm import tqdm

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '..')
from common import xcs_cosmo, m_norm, tx_norm, leave_one_jackknife

import xga
from xga.relations.fit import scaling_relation_lira

# Setting up this constant that controls the confidence intervals calculated for the scatter change
#  distributions - makes it much easier to change the exact value later if we need too. This is 
#  what is going to be used to define the scatter-dominating clusters.
CONF_PERC = 90

## Loading data files and relations

We load the 'base' SDSSRM-XCS sample file, containing some basic information about galaxy cluster names, positions, and redshifts. We also load the SDSSRM-XCS cluster property results files from the first paper of this series. 

The scaling relations which we are performing our bootstrapping scatter assessment on are also loaded in here.

### SDSSRM-XCS base sample

In [ ]:
sdssxcs_base = pd.read_csv("../../sample_files/SDSSRM-XCS_base_sample.csv")
sdssxcs_base.head(6)

Calculating E(z) values for these clusters:

In [ ]:
sdssxcs_base['E'] = xcs_cosmo.efunc(sdssxcs_base['z'].values)

### SDSSRM-XCS $T_{\rm{X}}$ and $L_{\rm{X}}$

In [ ]:
sdssxcs_txlx = pd.read_csv("../../sample_files/paper_one_results/sdssrm-xcs_txlx_v1.0.0.csv")
sdssxcs_txlx.head(6)

### SDSSRM-XCS masses

In [ ]:
sdssxcs_mass = pd.read_csv("../../sample_files/paper_one_results/sdssrm-xcs_mass_v1.0.0.csv")
sdssxcs_mass.head(6)

### Combining tables

In [ ]:
sdssxcs_samp = pd.merge(sdssxcs_base, sdssxcs_txlx, left_on='name', right_on='name', how='outer')
sdssxcs_samp = pd.merge(sdssxcs_samp, sdssxcs_mass, left_on='name', right_on='name', how='outer')

### $M^{\rm{tot}}_{500}$-$T_{\rm{X,500}}$

In [ ]:
with open('../../outputs/scaling_relations/sdssrm-xcs_new/mtot-tx/turner2025_mtot500_tx500.xgarel', 'rb') as scalo:
    mtot500_tx500 = pickle.load(scalo)

### $M^{\rm{tot}}_{500}$-$T_{\rm{X,500ce}}$

In [ ]:
with open('../../outputs/scaling_relations/sdssrm-xcs_new/mtot-tx/turner2025_mtot500_tx500ce.xgarel', 'rb') as scalo:
    mtot500_tx500ce = pickle.load(scalo)

### $M^{\rm{tot}}_{2500}$-$T_{\rm{X,2500}}$ 

In [ ]:
with open('../../outputs/scaling_relations/sdssrm-xcs_new/mtot-tx/turner2025_mtot2500_tx2500.xgarel', 'rb') as scalo:
    mtot2500_tx2500 = pickle.load(scalo)

## Searching for scatter-dominating-clusters using "leave-one-out" jack-knifing

The goal of this notebook is to measure a set of jackknifed scaling relations - this will be achieved by iteratively excluding every single cluster from the sample, one at a time, and fitting those sub-samples in the same was as the original scaling relation  - any large changes in normalisation, slope, or scatter (from the original values measured with whole sample, and from each other) of these relations will help to inform us which clusters (if any) are the dominant source of overall scatter in the relations.

### Setting up input samples

We want to make sure we're only using the clusters which have measurements of the properties we're interested in - **104** of the sample have an $M^{\rm{tot}}_{500}$ measurement, and **91** have a $M^{\rm{tot}}_{2500}$ measurement, for instance. To make this more sample selection more generally useful, as it will be used in other jack-knifing notebooks, we use $M_{\rm{gas}}$ measurements as the test for whether a cluster will be included in a particular sample or not.

#### Clusters with $R_{500}$ properties

In [ ]:
# This is the set of SDSSRM-XCS clusters with R500 gas masses - there will not be a hydrostatic mass if there is no gas mass, and
#  we will be using at least one type of mass in all the scaling relations we're assessing
sdssxcs_samp_with_gm500 = sdssxcs_samp[np.isfinite(sdssxcs_samp['Mg500_wraderr'])]

#### Clusters with $R_{2500}$ properties

In [ ]:
# This is the set of SDSSRM-XCS clusters with R2500 gas masses - there will not be a hydrostatic mass if there is no gas mass, and
#  we will be using at least one type of mass in all the scaling relations we're assessing
sdssxcs_samp_with_gm2500 = sdssxcs_samp[np.isfinite(sdssxcs_samp['Mg2500_wraderr'])]

### Assessing the $M^{\rm{tot}}_{500}$-$T_{\rm{X,500}}$ relation

In [ ]:
l1_mtot500_tx500_rels, l1_mtot500_tx500_res = leave_one_jackknife(sdssxcs_samp_with_gm500, mtot500_tx500)

One of the outputs of the jackknifing function is a dataframe containing the relation parameters linked to the name of the cluster that was excluded in each case - we save them to disk as well as making use of them later to measure scaling relations with the scatter-dominating clusters removed.

In [ ]:
l1_mtot500_tx500_res.to_csv("../../outputs/result_files/exploring_scatter/leave_one_jackknife/mtot-tx/"\
                            "mtot500_tx500_l1_jackknife_rel_pars.csv")
l1_mtot500_tx500_res.head(6)

#### Diagnostic distribution of percentage change of scatter

In [ ]:
plt.figure(figsize=(5, 5))

plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)

# Plotting the distribution of percentage change in scatter values
plt.hist(l1_mtot500_tx500_res['scatter_perc_change'].values, bins='auto', color='seagreen', alpha=0.65, 
         label=r"$\sigma$ change distribution", histtype='step', linewidth=3)

# Calculating the median and specified confidence intervals
cur_med = np.median(l1_mtot500_tx500_res['scatter_perc_change'].values)
cur_low = np.percentile(l1_mtot500_tx500_res['scatter_perc_change'].values, (50-(CONF_PERC/2)))
cur_upp = np.percentile(l1_mtot500_tx500_res['scatter_perc_change'].values, (50+(CONF_PERC/2)))

plt.axvline(cur_med, color='black', label='Median')
plt.axvline(cur_low, color='black', linestyle='dashed', label='{}% limits'.format(CONF_PERC))
plt.axvline(cur_upp, color='black', linestyle='dashed')


plt.title(r'$M^{\rm{tot}}_{500}$-$T_{\rm{X,500}}$ scatter change with jackknife')
plt.ylabel('N', fontsize=14)
plt.xlabel(r'$\dfrac{\sigma_{\rm{jackknife}}-\sigma_{\rm{original}}}{\sigma_{\rm{original}}}$ [%]', fontsize=14)

plt.legend()
plt.tight_layout()
plt.show()

print("Median percentage change in scatter of {v:.2f}%".format(v=cur_med))
print("{c}% confidence limits on percentage change in scatter of {l:.2f}% to "\
      "{u:.2f}%\n".format(c=CONF_PERC, l=cur_low, u=cur_upp))

#### Selecting 'scatter-dominating' clusters

In [ ]:
l1_mtot500_tx500_domin = l1_mtot500_tx500_res[l1_mtot500_tx500_res['scatter_perc_change'] < cur_low]
l1_mtot500_tx500_domin

#### Fitting a new version of the relation without the 'scatter-dominating' clusters

In [ ]:
cur_samp_wo_domin = sdssxcs_samp_with_gm500[~sdssxcs_samp_with_gm500['name'].isin(l1_mtot500_tx500_domin['dropped_cluster'].values)]

# Setting up property variables in astropy quantity objects
mtot500 = Quantity(cur_samp_wo_domin[['Mhy500_wraderr', 'Mhy500_wraderr-', 'Mhy500_wraderr+']].values*1e+14, 'Msun')\
    *cur_samp_wo_domin['E'].values[..., None]
tx500 = Quantity(cur_samp_wo_domin[['Tx_500', 'Tx_500-','Tx_500+']].values, 'keV')

mtot500_tx500_wo_domin = scaling_relation_lira(mtot500[:, 0], mtot500[:, 1:], tx500[:, 0], tx500[:, 1:], m_norm, tx_norm, 
                                               y_name=r"$E(z)M^{\rm{tot}}_{500}$", x_name=r"$T_{\rm{X,500}}$", 
                                               dim_hubb_ind=1, point_names=cur_samp_wo_domin['name'].values)
mtot500_tx500_wo_domin.model_colour = 'tab:cyan'
mtot500_tx500_wo_domin.author = 'Turner et al.'
mtot500_tx500_wo_domin.year = 2025
mtot500_tx500_wo_domin.name = r"Turner et al. '$\sigma$ dominating excluded' $E(z)M^{\rm{tot}}_{500}$-$T_{\rm{X,500}}$"

In [ ]:
print("Slope of {v:.3f} ± {e:.3f}\n".format(v=mtot500_tx500_wo_domin.pars[0][0], e=mtot500_tx500_wo_domin.pars[0][1]))
print("Normalisation of {v:.3f} ± {e:.3f}\n".format(v=mtot500_tx500_wo_domin.pars[1][0], e=mtot500_tx500_wo_domin.pars[1][1]))
print("Scatter of {v:.3f} ± {e:.3f}".format(v=mtot500_tx500_wo_domin.scatter_par[0], e=mtot500_tx500_wo_domin.scatter_par[1]))

### Assessing the $M^{\rm{tot}}_{500}$-$T_{\rm{X,500ce}}$ relation

In [ ]:
l1_mtot500_tx500ce_rels, l1_mtot500_tx500ce_res = leave_one_jackknife(sdssxcs_samp_with_gm500, mtot500_tx500ce, 
                                                                      y_cols=['Tx_500ce', 'Tx_500ce-', 'Tx_500ce+'],
                                                                      y_name=r"$T_{\rm{X,500ce}}$")

One of the outputs of the jackknifing function is a dataframe containing the relation parameters linked to the name of the cluster that was excluded in each case - we save them to disk as well as making use of them later to measure scaling relations with the scatter-dominating clusters removed.

In [ ]:
l1_mtot500_tx500ce_res.to_csv("../../outputs/result_files/exploring_scatter/leave_one_jackknife/mtot-tx/"\
                              "mtot500_tx500ce_l1_jackknife_rel_pars.csv")
l1_mtot500_tx500ce_res.head(6)

#### Diagnostic distribution of percentage change of scatter

In [ ]:
plt.figure(figsize=(5, 5))

plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)

# Plotting the distribution of percentage change in scatter values
plt.hist(l1_mtot500_tx500ce_res['scatter_perc_change'].values, bins='auto', color='seagreen', alpha=0.65, 
         label=r"$\sigma$ change distribution", histtype='step', linewidth=3)

# Calculating the median and specified confidence intervals
cur_med = np.median(l1_mtot500_tx500ce_res['scatter_perc_change'].values)
cur_low = np.percentile(l1_mtot500_tx500ce_res['scatter_perc_change'].values, (50-(CONF_PERC/2)))
cur_upp = np.percentile(l1_mtot500_tx500ce_res['scatter_perc_change'].values, (50+(CONF_PERC/2)))

plt.axvline(cur_med, color='black', label='Median')
plt.axvline(cur_low, color='black', linestyle='dashed', label='{}% limits'.format(CONF_PERC))
plt.axvline(cur_upp, color='black', linestyle='dashed')


plt.title(r'$M^{\rm{tot}}_{500}$-$T_{\rm{X,500ce}}$ scatter change with jackknife')
plt.ylabel('N', fontsize=14)
plt.xlabel(r'$\dfrac{\sigma_{\rm{jackknife}}-\sigma_{\rm{original}}}{\sigma_{\rm{original}}}$ [%]', fontsize=14)

plt.legend()
plt.tight_layout()
plt.show()

print("Median percentage change in scatter of {v:.2f}%".format(v=cur_med))
print("{c}% confidence limits on percentage change in scatter of {l:.2f}% to "\
      "{u:.2f}%\n".format(c=CONF_PERC, l=cur_low, u=cur_upp))

#### Selecting 'scatter-dominating' clusters

In [ ]:
l1_mtot500_tx500ce_domin = l1_mtot500_tx500ce_res[l1_mtot500_tx500ce_res['scatter_perc_change'] < cur_low]
l1_mtot500_tx500ce_domin

#### Fitting a new version of the relation without the 'scatter-dominating' clusters

In [ ]:
cur_samp_wo_domin = sdssxcs_samp_with_gm500[~sdssxcs_samp_with_gm500['name'].isin(l1_mtot500_tx500ce_domin['dropped_cluster'].values)]

# Setting up property variables in astropy quantity objects
mtot500 = Quantity(cur_samp_wo_domin[['Mhy500_wraderr', 'Mhy500_wraderr-', 'Mhy500_wraderr+']].values*1e+14, 'Msun')\
    *cur_samp_wo_domin['E'].values[..., None]
tx500ce = Quantity(cur_samp_wo_domin[['Tx_500ce', 'Tx_500ce-','Tx_500ce+']].values, 'keV')

mtot500_tx500ce_wo_domin = scaling_relation_lira(mtot500[:, 0], mtot500[:, 1:], tx500ce[:, 0], tx500ce[:, 1:], m_norm, tx_norm, 
                                                 y_name=r"$E(z)M^{\rm{tot}}_{500}$", x_name=r"$T_{\rm{X,500ce}}$", 
                                                 dim_hubb_ind=1, point_names=cur_samp_wo_domin['name'].values)
mtot500_tx500ce_wo_domin.model_colour = 'steelblue'
mtot500_tx500ce_wo_domin.author = 'Turner et al.'
mtot500_tx500ce_wo_domin.year = 2025
mtot500_tx500ce_wo_domin.name = r"Turner et al. '$\sigma$ dominating excluded' $E(z)M^{\rm{tot}}_{500ce}$-$T_{\rm{X,500}}$"

In [ ]:
print("Slope of {v:.3f} ± {e:.3f}\n".format(v=mtot500_tx500ce_wo_domin.pars[0][0], e=mtot500_tx500ce_wo_domin.pars[0][1]))
print("Normalisation of {v:.3f} ± {e:.3f}\n".format(v=mtot500_tx500ce_wo_domin.pars[1][0], e=mtot500_tx500ce_wo_domin.pars[1][1]))
print("Scatter of {v:.3f} ± {e:.3f}".format(v=mtot500_tx500ce_wo_domin.scatter_par[0], e=mtot500_tx500ce_wo_domin.scatter_par[1]))

### Assessing the $M^{\rm{tot}}_{2500}$-$T_{\rm{X,2500}}$ relation

In [ ]:
l1_mtot2500_tx2500_rels, l1_mtot2500_tx2500_res = leave_one_jackknife(sdssxcs_samp_with_gm500, mtot2500_tx2500)

One of the outputs of the jackknifing function is a dataframe containing the relation parameters linked to the name of the cluster that was excluded in each case - we save them to disk as well as making use of them later to measure scaling relations with the scatter-dominating clusters removed.

In [ ]:
l1_mtot2500_tx2500_res.to_csv("../../outputs/result_files/exploring_scatter/leave_one_jackknife/mtot-tx/"\
                            "mtot2500_tx2500_l1_jackknife_rel_pars.csv")
l1_mtot2500_tx2500_res.head(6)

#### Diagnostic distribution of percentage change of scatter

In [ ]:
plt.figure(figsize=(5, 5))

plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)

# Plotting the distribution of percentage change in scatter values
plt.hist(l1_mtot2500_tx2500_res['scatter_perc_change'].values, bins='auto', color='seagreen', alpha=0.65, 
         label=r"$\sigma$ change distribution", histtype='step', linewidth=3)

# Calculating the median and specified confidence intervals
cur_med = np.median(l1_mtot2500_tx2500_res['scatter_perc_change'].values)
cur_low = np.percentile(l1_mtot2500_tx2500_res['scatter_perc_change'].values, (50-(CONF_PERC/2)))
cur_upp = np.percentile(l1_mtot2500_tx2500_res['scatter_perc_change'].values, (50+(CONF_PERC/2)))

plt.axvline(cur_med, color='black', label='Median')
plt.axvline(cur_low, color='black', linestyle='dashed', label='{}% limits'.format(CONF_PERC))
plt.axvline(cur_upp, color='black', linestyle='dashed')

plt.title(r'$M^{\rm{tot}}_{2500}$-$T_{\rm{X,2500}}$ scatter change with jackknife')
plt.ylabel('N', fontsize=14)
plt.xlabel(r'$\dfrac{\sigma_{\rm{jackknife}}-\sigma_{\rm{original}}}{\sigma_{\rm{original}}}$ [%]', fontsize=14)

plt.legend()
plt.tight_layout()
plt.show()

print("Median percentage change in scatter of {v:.2f}%".format(v=cur_med))
print("{c}% confidence limits on percentage change in scatter of {l:.2f}% to "\
      "{u:.2f}%\n".format(c=CONF_PERC, l=cur_low, u=cur_upp))

#### Selecting 'scatter-dominating' clusters

In [ ]:
l1_mtot2500_tx2500_domin = l1_mtot2500_tx2500_res[l1_mtot2500_tx2500_res['scatter_perc_change'] < cur_low]
l1_mtot2500_tx2500_domin

#### Fitting a new version of the relation without the 'scatter-dominating' clusters

In [ ]:
cur_samp_wo_domin = sdssxcs_samp_with_gm500[~sdssxcs_samp_with_gm500['name'].isin(l1_mtot2500_tx2500_domin['dropped_cluster'].values)]

# Setting up property variables in astropy quantity objects
mtot2500 = Quantity(cur_samp_wo_domin[['Mhy2500_wraderr', 'Mhy2500_wraderr-', 'Mhy2500_wraderr+']].values*1e+14, 'Msun')\
    *cur_samp_wo_domin['E'].values[..., None]
tx2500 = Quantity(cur_samp_wo_domin[['Tx_2500', 'Tx_2500-','Tx_2500+']].values, 'keV')

mtot2500_tx2500_wo_domin = scaling_relation_lira(mtot2500[:, 0], mtot2500[:, 1:], tx2500[:, 0], tx2500[:, 1:], m_norm, tx_norm, 
                                                 y_name=r"$E(z)M^{\rm{tot}}_{2500}$", x_name=r"$T_{\rm{X,2500}}$", 
                                                 dim_hubb_ind=1, point_names=cur_samp_wo_domin['name'].values)
mtot2500_tx2500_wo_domin.model_colour = 'navajowhite'
mtot2500_tx2500_wo_domin.author = 'Turner et al.'
mtot2500_tx2500_wo_domin.year = 2025
mtot2500_tx2500_wo_domin.name = r"Turner et al. '$\sigma$ dominating excluded' $E(z)M^{\rm{tot}}_{2500}$-$T_{\rm{X,2500}}$"

In [ ]:
print("Slope of {v:.3f} ± {e:.3f}\n".format(v=mtot2500_tx2500_wo_domin.pars[0][0], e=mtot2500_tx2500_wo_domin.pars[0][1]))
print("Normalisation of {v:.3f} ± {e:.3f}\n".format(v=mtot2500_tx2500_wo_domin.pars[1][0], e=mtot2500_tx2500_wo_domin.pars[1][1]))
print("Scatter of {v:.3f} ± {e:.3f}".format(v=mtot2500_tx2500_wo_domin.scatter_par[0], e=mtot2500_tx2500_wo_domin.scatter_par[1]))